In [1]:
#!/usr/bin/env python3
import requests#
import pandas as pd
import geopandas as gpd
import shapely


In [7]:
url = "https://www.smard.de/home/downloadcenter/download-kraftwerksdaten/?downloadAttributes=%7B%22selectedPowerPlant%22:%2211WD2SUE2000103B%22,%22selectedContent%22:%22generation%22,%22from%22:1714514400000,%22to%22:1722463199999,%22selectedFileType%22:%22CSV%22%7D"

In [11]:
headers = {
            "Accept": "application/json"
        }

response = requests.get(url, headers=headers, params=None)

In [13]:
response

<Response [200]>

In [2]:
class PublicApiClient:
    BASE_URL = "https://app.thru.de//api/v1/prtr"

    def __init__(self, api_key: str = None):
        self.headers = {
            "Accept": "application/json"
        }
        if api_key:
            self.headers["Authorization"] = f"Bearer {api_key}"

    def _get(self, endpoint: str, params: dict = None):
        url = f"{self.BASE_URL}/{endpoint}"
        response = requests.get(url, headers=self.headers, params=params)
        self._handle_response(response)
        return response.json()

    def _handle_response(self, response):
        if not response.ok:
            raise Exception(f"API Error {response.status_code}: {response.text}")
        
    def get_federal_states(self):
        """Fetches the list of federal states."""
        return self._get("federal_states")
    
    def get_facilities_by_year_and_state(self, year, state):
        """Fetches facilities for a given reporting year and federal state."""
        endpoint = f"years/{year}/federal_states/{state}"
        return self._get(endpoint)
    
    def get_facility_details(self, year, state, facility_id):
        """Fetches details of a specific facility."""
        endpoint = f"years/{year}/federal_states/{state}/facilities/{facility_id}"
        return self._get(endpoint)
    
    def get_plants_by_year_and_state(self, year, state):
        """Fetches plants for a given reporting year, federal state, and facility."""
        endpoint = f"years/{year}/federal_states/{state}"
        return self._get(endpoint)

In [3]:
year = 2021
state = "BY"  # State code for Bavaria

d = PublicApiClient()
plants = d.get_plants_by_year_and_state(2021, "BY")
facility_ids = [plant['facility_identifier'] for plant in plants]


prtr_data = pd.DataFrame()

for id in facility_ids:
    data = d.get_facility_details(year, state, id)
    
    # only process facilities that release air pollutants
    if data['release_to_air'] == []:
        continue
    
    else:
        name = data['name']
        year = data['reporting_year']
        location = shapely.wkt.loads(str(data['location']).split(';')[1])
        
        # construct a DataFrame for the facility
        release = pd.json_normalize(data['release_to_air'])
        release['facility_identifier'] = id
        release['facility_name'] = name
        release['reporting_year'] = year
        release['location'] = location
    
        prtr_data = pd.concat([prtr_data, release], ignore_index=True)
        
        print(f"Processed facility {id} - {name}")


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00381 - Schindlbeck GbR
Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00400 - RW silicium GmbH, Werk Pocking


KeyboardInterrupt: 

In [5]:
data

{'facility_identifier': 'https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00394',
 'inspire_id': 'https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00394',
 'thematic_id': '06-09-275-9001-0001',
 'reporting_year': '2021',
 'federal_state': 'BY',
 'name': 'Galvano-Tec GmbH',
 'address': {'street': 'Passauer Str.',
  'street_number': '36',
  'zip_code': '94130',
  'city': 'Obernzell'},
 'river_basin_district': 'Danube',
 'parent_company': 'Galvano-Tec GmbH',
 'owner': None,
 'operator': None,
 'operator_information_for_general_public': None,
 'nace_code': '25.61',
 'nace_industry': 'Treatment and coating of metals',
 'location': 'SRID=4326;POINT (13.628019703562558 48.558337161945204)',
 'installation_count': 1,
 'operating_hours': None,
 'main_activity': {'activity': {'name': 'Installations for surface treatment of metals and plastic materials using an electrolytic or chemical process',
   'sector': 'Metal industry'},
  'confidentiality': []},
 'secondary_activities': [],
 'relea